In [1]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

C:\Users\Grego\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
df = pd.read_csv('creditcard.csv')

In [3]:
df.head(2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0


In [4]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [5]:
X = df.drop(['Class'],axis=1).values
y = df['Class'].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
smt = SMOTE()
X_train, y_train = smt.fit_sample(X_train, y_train)

In [8]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [24]:
model = Sequential()

model.add(Dense(50, kernel_initializer = "uniform",activation = "relu", input_dim=30))
model.add(Dropout(rate = 0.1))
model.add(Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))
model.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

model.fit(X_train, y_train, batch_size = 36, epochs = 40)

Epoch 1/40
398014/398014 [==============================] - 16s 39us/step - loss: 0.0564 - acc: 0.9793
Epoch 2/40
398014/398014 [==============================] - 14s 36us/step - loss: 0.0216 - acc: 0.9931
Epoch 3/40
398014/398014 [==============================] - 14s 36us/step - loss: 0.0138 - acc: 0.9962
Epoch 4/40
398014/398014 [==============================] - 15s 38us/step - loss: 0.0104 - acc: 0.9973
Epoch 5/40
398014/398014 [==============================] - 15s 38us/step - loss: 0.0084 - acc: 0.9979
Epoch 6/40
398014/398014 [==============================] - 15s 38us/step - loss: 0.0076 - acc: 0.9982
Epoch 7/40
398014/398014 [==============================] - 15s 39us/step - loss: 0.0068 - acc: 0.9985
Epoch 8/40
398014/398014 [==============================] - 15s 37us/step - loss: 0.0062 - acc: 0.9986
Epoch 9/40
398014/398014 [==============================] - 16s 40us/step - loss: 0.0059 - acc: 0.9987
Epoch 10/40
398014/398014 [==============================] - 15s 39us/ste

In [25]:
y_pred = model.predict(X_test)

In [26]:
y_pred = (y_pred > 0.5)

In [27]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[85261    47]
 [   22   113]]


In [28]:
cr = classification_report(y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85308
           1       0.71      0.84      0.77       135

    accuracy                           1.00     85443
   macro avg       0.85      0.92      0.88     85443
weighted avg       1.00      1.00      1.00     85443



In [14]:
def create_model():
    model = Sequential()
    
    model.add(Dense(50, kernel_initializer = "uniform",activation = "relu", input_dim=30))
    model.add(Dropout(rate = 0.1))
    model.add(Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))
    model.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])
    
    return model

In [15]:
model = KerasClassifier(build_fn=create_model, epochs=40, verbose=0)

In [16]:
params = {
    'batch_size':[33, 36]
}

In [17]:
grid = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1)

In [18]:
grid_result = grid.fit(X_train,y_train)

In [19]:
best_param = grid_result.best_params_

In [20]:
best_param

{'batch_size': 36}

In [21]:
grid_result.cv_results_

{'mean_fit_time': array([908.68625617, 584.81757887]),
 'std_fit_time': array([  1.63645536, 185.63319497]),
 'mean_score_time': array([5.37358077, 3.86842934]),
 'std_score_time': array([0.50593579, 1.98703101]),
 'param_batch_size': masked_array(data=[33, 36],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'batch_size': 33}, {'batch_size': 36}],
 'split0_test_score': array([0.99849252, 0.99874126]),
 'split1_test_score': array([0.9996985 , 0.99966835]),
 'split2_test_score': array([0.99973619, 1.        ]),
 'mean_test_score': array([0.99930907, 0.99946987]),
 'std_test_score': array([0.00057759, 0.0005327 ]),
 'rank_test_score': array([2, 1])}

In [22]:
grid_result.best_score_

0.9994698676413075